In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Read data from file
with open('data.txt', 'r') as f:
    data = f.readlines()

# Define player data
players = []
for i in range(0, len(data), 3):
    player = {
        'defense_ranks': np.array([int(j) for j in data[i].split(',')], dtype=np.float32),
        'fantasy_scores': np.array([float(j) for j in data[i+1].split(',')], dtype=np.float32),
        'skill_score': np.array([float(j) for j in data[i+2].split(',')], dtype=np.float32),
    }
    players.append(player)

# Process inputs and targets for each player
X_train, Y_train, X_test, Y_test = [], [], [], []
 
for player in players[:16]:
    player_inputs = torch.from_numpy(player['defense_ranks']).view(1, 15, 1).float()
    player_inputs = torch.cat((player_inputs, torch.from_numpy(player['skill_score']).view(1, 15, 1).float()), dim=2)
    X_train.append(player_inputs)
    Y_train.append(torch.from_numpy(player['fantasy_scores']).view(1, 15, 1).float())
  
for player in players[-4:]:
    player_test_inputs = torch.from_numpy(player['defense_ranks']).view(1, 15, 1).float()
    player_test_inputs = torch.cat((player_test_inputs, torch.from_numpy(player['skill_score']).view(1, 15, 1).float()), dim=2)
    X_test.append(player_test_inputs)  
    Y_test.append(torch.from_numpy(player['fantasy_scores']).view(1, 15, 1).float())

# Combine inputs and targets for all players
X_train = torch.cat(X_train, dim=0)
Y_train = torch.cat(Y_train, dim=0)
X_test = torch.cat(X_test, dim=0)
Y_test = torch.cat(Y_test, dim=0)

In [562]:
# Define the neural network architecture
class RegressionModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

# Set hyperparameters
input_size = 2
hidden_size = 50
learning_rate = 0.0001

# Create the model
model = RegressionModel(input_size, hidden_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [751]:
# Training loop for each player with batch size of 2
batch_size = 8
num_players = len(X_train)
num_batches = num_players // batch_size # Only full batches
num_epochs = 50

for batch in range(num_batches):
    batch_inputs = X_train[batch * batch_size: (batch + 1) * batch_size]
    batch_targets = Y_train[batch * batch_size: (batch + 1) * batch_size]

    # Training loop
    for epoch in range(num_epochs):
        # Forward pass
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print the loss for every epoch
        if epoch + 1 == num_epochs:
            print(f"Batch {batch+1}/{num_batches}, Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

# Make predictions for the player's test inputs
with torch.no_grad():
    player_predicted = model(X_test)

print() # blank line
# Print the predictions and original scores for the player
for i in range(len(X_test[0])):
    for j in range(len(X_test)):
        print(f"Player {j+1}: Skill Score: {round(X_test[j][i][1].item(), 2)}, Defense Rank: {X_test[j][i][0]}, " + 
              f"Predicted Fantasy Score: {round(player_predicted[j][i].item(), 2)}, Actual Fantasy Score: {round(Y_test[j][i].item(), 2)}")
    print() # blank line

Batch 1/2, Epoch 50/50, Loss: 59.86751937866211
Batch 2/2, Epoch 50/50, Loss: 77.42931365966797

Player 1: Skill Score: 13.54, Defense Rank: 26.0, Predicted Fantasy Score: 12.17, Actual Fantasy Score: 4.9
Player 2: Skill Score: 14.92, Defense Rank: 30.0, Predicted Fantasy Score: 12.31, Actual Fantasy Score: 4.6
Player 3: Skill Score: 10.97, Defense Rank: 2.0, Predicted Fantasy Score: 10.78, Actual Fantasy Score: 13.7
Player 4: Skill Score: 10.78, Defense Rank: 3.0, Predicted Fantasy Score: 11.19, Actual Fantasy Score: 5.0

Player 1: Skill Score: 13.54, Defense Rank: 17.0, Predicted Fantasy Score: 11.95, Actual Fantasy Score: 4.9
Player 2: Skill Score: 14.92, Defense Rank: 9.0, Predicted Fantasy Score: 16.03, Actual Fantasy Score: 5.3
Player 3: Skill Score: 10.97, Defense Rank: 16.0, Predicted Fantasy Score: 10.53, Actual Fantasy Score: 7.4
Player 4: Skill Score: 10.78, Defense Rank: 12.0, Predicted Fantasy Score: 11.65, Actual Fantasy Score: 13.6

Player 1: Skill Score: 13.54, Defense 